# 「LSTM単語言語モデルの動的量子化（ベータ版）」

【原題】(beta) Dynamic Quantization on an LSTM Word Language Model

【原著】[James Reed](https://github.com/jamesr66a)

【編著】[Seth Weidman](https://github.com/SethHWeidman/)

【元URL】https://pytorch.org/tutorials/advanced/dynamic_quantization_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年1月31日

【チュトーリアル概要】

本チュートリアルでは、重みの数値を格納するビット数を減らし、モデルサイズの削減、推論速度向上に寄与する量子化を、LSTMベースの次単語予測モデルに適用します

## 導入

量子化では、モデルの重みと活性化をfloatからintに変換します。

量子化により、精度への影響を最小限に抑えた上でモデルのサイズを小さくし、推論を高速化することができます。

本チュートリアルでは、PyTorchのサンプル例より、[単語言語モデル](https://github.com/pytorch/examples/tree/master/word_language_model)に対して、LSTMベースの次単語予測モデルに最も簡単な量子化の仕組みである[動的量子化](https://pytorch.org/docs/stable/quantization.html#torch.quantization.quantize_dynamic) を適用します。

In [ ]:
%matplotlib inline

In [ ]:
# インポート
import os
from io import open
import time

import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. モデルの定義

単語言語モデルの例より、こちらの[モデル](https://github.com/pytorch/examples/blob/master/word_language_model/model.py)に従い、LSTMモデルの構造を定義します。

In [ ]:
class LSTMModel(nn.Module):
    """エンコーダー、再帰モジュール、そしてデコーダーを含むモデル構成"""

    def __init__(self, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(LSTMModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output)
        return decoded, hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                weight.new_zeros(self.nlayers, bsz, self.nhid))

## 2. テキストデータの読み込み

同様に、単語言語モデルの[前処理](https://github.com/pytorch/examples/blob/master/word_language_model/data.py)に従い、[Wikitext-2 データセット](https://www.google.com/search?q=wikitext+2+data)を`Corpus`に読み込みます。

In [ ]:
# 日本語訳版追加　Wikitext-2データセットのダウンロード
import urllib.request
import zipfile

# ダウンロード
for item in ["train.txt", "valid.txt", "test.txt"]:
    url = "https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2/"+item
    save_path = "/content/data/wikitext-2/"+item
    urllib.request.urlretrieve(url, save_path)

# 以下のモデルはこのチュートリアルとは、モデルサイズが異なる・・・
#url ="https://s3.amazonaws.com/pytorch-tutorial-assets/word_language_model_quantize.pth"
#save_path = "/content/data/word_language_model_quantize.pth"
#urllib.request.urlretrieve(url, save_path)


('/content/data/word_language_model_quantize.pth',
 <http.client.HTTPMessage at 0x7f9d138c1cf8>)

In [ ]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """テキストファイルのトークン化"""
        assert os.path.exists(path)
        # 辞書に単語を追加
        with open(path, 'r', encoding="utf8") as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)

        # ファイルの内容をトークン化
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)

        return ids

#model_data_filepath = 'data/'
model_data_filepath = '/content/data/'  # 日本語訳版変更、GoogleClabで実行できるパスに 

corpus = Corpus(model_data_filepath + 'wikitext-2')

## 3. 事前訓練済みモデルの読み込み

本チュートリアルでは動的量子化について扱いますが、量子化のテクニックはモデルの訓練が完了したあとに適用します。

そのため、まずは単に事前訓練済みの重みをモデルの構造に読み込みます。

なお、使用する重みは単語言語モデルの例のデフォルトの設定を使って、5エポックにわたり訓練して求めた値です。

In [ ]:
ntokens = len(corpus.dictionary)

model = LSTMModel(
    ntoken = ntokens,
    ninp = 512,
    nhid = 256,
    nlayers = 5,
)

# 日本語訳版変更　今回は訓練済みデータの読み込みはなしにする。該当ファイルが見つからない・・・
#model.load_state_dict(
#    torch.load(
#        model_data_filepath + 'word_language_model_quantize.pth',
#        map_location=torch.device('cpu')
#        )
#    )

model.eval()
print(model)

LSTMModel(
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(2363, 512)
  (rnn): LSTM(512, 256, num_layers=5, dropout=0.5)
  (decoder): Linear(in_features=256, out_features=2363, bias=True)
)


前と同じように[こちら](https://github.com/pytorch/examples/blob/master/word_language_model/generate.py)を参考にして文章を生成し、訓練済みモデルが適切に動作することを確かめます。

In [ ]:
input_ = torch.randint(ntokens, (1, 1), dtype=torch.long)
hidden = model.init_hidden(1)
temperature = 1.0
num_words = 1000

with open(model_data_filepath + 'out.txt', 'w') as outf:
    with torch.no_grad():  # 勾配のトラッキングを無効化
        for i in range(num_words):
            output, hidden = model(input_, hidden)
            word_weights = output.squeeze().div(temperature).exp().cpu()
            word_idx = torch.multinomial(word_weights, 1)[0]
            input_.fill_(word_idx)

            word = corpus.dictionary.idx2word[word_idx]

            outf.write(str(word.encode('utf-8')) + ('\n' if i % 20 == 19 else ' '))

            if i % 100 == 0:
                print('| Generated {}/{} words'.format(i, 1000))

with open(model_data_filepath + 'out.txt', 'r') as outf:
    all_output = outf.read()
    print(all_output)

| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words
b'&quot;16822470375&quot;,' b'or' b'data-menu-item="i3actions-tab"' b'&quot;control&quot;},' b'class="jump-to-octicon' b'or' b'js-permalink-shortcut"' b'content="ZzhVyEFwb7w3e0-uOTltm8Jsck2F5StVihD0exw2fsA">' b'Open' b'data-src="https://github.githubassets.com/assets/chunk-runner-groups-13e1fec0.js"></script>' b'data-platforms="windows,mac"' b'left-0' b'data-hydro-click-hmac="ef93c2505f3805730378a168ba4cd6a5c4869e59513f8ab970098de8673b7e91"' b'cursor-pointer"' b'class="Bump-link-symbol' b'pt-lg-3">' b'js-jump-to-global-search' b'class="css-truncate' b'starred' b'flex-justify-between'
b'Box-body' b'd="M1.643' b'd="M11.5' b'css-truncate' b'15H1.918a1.75' b'5.169.88' b'01-.765' b'logged' b'01-1.482-.236A5.507' b'can\xe2\x80\x

GPT-2ではありませんが、モデルが言語の構造を学び始めていることが確認できます。

これで動的量子化を解説する準備がほとんどできました。
あとはいくつか補助的な関数を定義するだけです。

In [ ]:
bptt = 25
criterion = nn.CrossEntropyLoss()
eval_batch_size = 1

# テストデータセットの作成
def batchify(data, bsz):
    # データセットをbsz個のパーツに綺麗に分割する方法を示します。
    nbatch = data.size(0) // bsz
    # うまく収まらない余分な要素を切り取ります。
    data = data.narrow(0, 0, nbatch * bsz)
    # bsz個のバッチ間でデータが均等になるように分割します。
    return data.view(bsz, -1).t().contiguous()

test_data = batchify(corpus.test, eval_batch_size)

# 評価関数
def get_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

def repackage_hidden(h):
  """隠れ層の状態を新しいテンソルにラップし、勾配の履歴から切り離します。"""

  if isinstance(h, torch.Tensor):
    return h.detach()
  else:
    return tuple(repackage_hidden(v) for v in h)

def evaluate(model_, data_source):
    # ドロップアウトを無効化する評価モードの有効化
    model_.eval()
    total_loss = 0.
    hidden = model_.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model_(data, hidden)
            hidden = repackage_hidden(hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

## 4. 動的量子化の検証

最後に`torch.quantization.quantize_dynamic`をモデルに呼び出します。

そして、以下の2点を指定します。
- モデル内の `nn.LSTM` モジュールと `nn.Linear` モジュールが量子化されるようにします。
- 重みが`int8`の値に変換されるようにします。

In [ ]:
import torch.quantization

quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.LSTM, nn.Linear}, dtype=torch.qint8
)
print(quantized_model)

LSTMModel(
  (drop): Dropout(p=0.5, inplace=False)
  (encoder): Embedding(2363, 512)
  (rnn): DynamicQuantizedLSTM(512, 256, num_layers=5, dropout=0.5)
  (decoder): DynamicQuantizedLinear(in_features=256, out_features=2363, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)


モデルは変わっていないように見えます。
では、量子化はどのような利点をもたらしたのでしょうか。

一つは、モデルのサイズが大幅に削減されている点です。

In [ ]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 18.850932
Size (MB): 8.38777


二つ目は、評価用の損失に影響をほとんど与えることなく、推論時間を短縮している点です。

注釈：量子化モデルは単一スレッドで実行されるため、単一スレッドでの比較を行うためにスレッド数を1つにしています。

In [ ]:
torch.set_num_threads(1)

def time_model_evaluation(model, test_data):
    s = time.time()
    loss = evaluate(model, test_data)
    elapsed = time.time() - s
    print('''loss: {0:.3f}\nelapsed time (seconds): {1:.1f}'''.format(loss, elapsed))

time_model_evaluation(model, test_data)
time_model_evaluation(quantized_model, test_data)

loss: 7.769
elapsed time (seconds): 3.2
loss: 7.769
elapsed time (seconds): 1.7


量子化せずにローカルのMacBook Pro上で上記コードを実行した場合、推論には200秒ほど掛かりますが、量子化を行うことで100秒ほどで推論を行えるようになります。

## 結論

動的量子化は、モデルサイズを削減する上で最も簡単な方法であると考えられ、また量子化によって発生する精度への影響は限定的なものです。

本チュートリアルに目を通していただき、ありがとうございます。
どんなフィードバックでも歓迎するので、もし何かございましたら、[こちら](https://github.com/pytorch/pytorch/issues)にissueを作成してください。（日本語訳注：日本語版チュートリアルに関連する内容であれば、是非[こちら](https://github.com/YutaroOgawa/pytorch_tutorials_jp)にissueの登録をお願いいたします。）